<a href="https://colab.research.google.com/github/iEpsilon-FPS/QU-MLinF/blob/master/5.4%20AutoML_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Module: Data Science in Finance, AutoML 
# Version 1.0
# Topic : AutoML - TPOT
# Example source: https://www.kaggle.com/wendykan/lending-club-loan-data
#####################################################################
# For support or questions, contact QuantUniversity at
# sri@quantuniversity.com
# Copyright 2020 QuantUniversity LLC.
#####################################################################

# AutoML with TPOT

AutoML is the process of automating an end-to-end Machine Learning pipeline. [TPOT](https://epistasislab.github.io/tpot/) specifically uses genetic programming to optimise these piplines by selecting the best model and its hyperparamters.

![TPOT](https://raw.githubusercontent.com/EpistasisLab/tpot/master/images/tpot-ml-pipeline.png)
<center>Image source: https://epistasislab.github.io/tpot/</center>

This notebook explains the basic workflow involved in an AutoML pipeline with TPOT

### Imports

In [ ]:
# for numerical analysis and data processing
import numpy as np
import pandas as pd

#AutoML
from sklearn.metrics.scorer import make_scorer

!pip install TPOT
from tpot import TPOTRegressor

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


     |████████████████████████████████| 92kB 5.9MB/s 
     |████████████████████████████████| 163kB 11.7MB/s 
  Created wheel for stopit: filename=stopit-1.1.2-cp36-none-any.whl size=11956 sha256=8090887003a74753ab4414c8cf83a610598b279d1b2b632c61d93e29cc653962
  Stored in directory: /root/.cache/pip/wheels/3c/85/2b/2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7
Successfully built stopit


In [ ]:
import requests
from io import StringIO

### Dataset

The data set is the lending data for lendingclub from August 2011 to December 2011 for some borrowers. The feature descriptions for the data are also provided. Not all the features are required for making predictions, some features are redundant in the original data file. The provided data file is already cleaned and only relevant features are provided. There are two types of features, numerical and categorical.

Reading the input data from csv file.

In [ ]:
orig_url_data='https://drive.google.com//file//d//1yG-JxC1Br3c8u3cfmKQWC9pgz6Pqggw5//view?usp=sharing'
file_id = orig_url_data.split('//')[-2]
dwn_url='https://drive.google.com//uc?export=download&id=' + file_id
url = requests.get(dwn_url).text
csv_raw = StringIO(url)
df = pd.read_csv(csv_raw)

orig_url_description='https://drive.google.com//file//d//1HFd4gKbknC28rHTWysec48NqfB6g3ZHx//view?usp=sharing'
file_id = orig_url_description.split('//')[-2]
dwn_url='https://drive.google.com//uc?export=download&id=' + file_id
url = requests.get(dwn_url).text
csv_raw = StringIO(url)
df_description = pd.read_excel(dwn_url)


del df['issue_d'] # removing issue date as it wont affect the prediction (redundant feature)

In [ ]:
df.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,loan_status_Binary
0,5000,36 months,10.65,162.87,B,B2,10+ years,RENT,24000.0,Verified,credit_card,AZ,27.65,0,1,0
1,2500,60 months,15.27,59.83,C,C4,< 1 year,RENT,30000.0,Source Verified,car,GA,1.00,0,5,1
2,2400,36 months,15.96,84.33,C,C5,10+ years,RENT,12252.0,Not Verified,small_business,IL,8.72,0,2,0
3,10000,36 months,13.49,339.31,C,C1,10+ years,RENT,49200.0,Source Verified,other,CA,20.00,0,1,0
4,3000,60 months,12.69,67.79,B,B5,1 year,RENT,80000.0,Source Verified,other,OR,17.94,0,0,0


### Preparing categorical features - One Hot Encoding

**Current version of TPOT does not support sparse matrix [link](https://github.com/EpistasisLab/tpot/issues/526), hence we need to do a bit of data preprocessing like converting categorical features.**

One way of representing categrical features is called one-hot encoding. Assume a categorical feature X with possible values as \[a, b, c, d\]. If in some sample the value of X=c, in one hot encoding ths particular feature is represented as X=\[0, 0, 1, 0\]. Its a binary array representation of length equal to the number of possible feature value, with 1 for the actual value.

If X can have values a b c d, then

X=c  
X=\[0, 0, 1, 0\]

X=a  
X=\[1, 0, 0, 0\]

In [ ]:
numeric_columns = df.select_dtypes(include=['float64','int64']).columns
categorical_columns = df.select_dtypes(include=['object']).columns

In [ ]:
for col in categorical_columns:
    df[col] = df[col].astype('category')

#### Dictionary for categorical features.

In [ ]:
categories={}
for cat in categorical_columns:
    categories[cat] = df[cat].cat.categories.tolist()

In [ ]:
p_categories = df['purpose'].cat.categories.tolist()
s_categories = df['addr_state'].cat.categories.tolist()
df[categorical_columns] = df[categorical_columns].apply(lambda x: x.cat.codes)

In [ ]:
df.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,loan_status_Binary
0,5000,0,10.65,162.87,1,6,1,2,24000.0,2,1,3,27.65,0,1,0
1,2500,1,15.27,59.83,2,13,10,2,30000.0,1,0,10,1.00,0,5,1
2,2400,0,15.96,84.33,2,14,1,2,12252.0,0,10,12,8.72,0,2,0
3,10000,0,13.49,339.31,2,10,1,2,49200.0,1,8,4,20.00,0,1,0
4,3000,1,12.69,67.79,1,9,0,2,80000.0,1,8,31,17.94,0,0,0


Storing interest rate statistics

In [ ]:
min_rate= df['int_rate'].min()
max_rate= df['int_rate'].max()
print(min_rate, max_rate, max_rate- min_rate)

5.42 24.11 18.689999999999998


In [ ]:
df_max = df.max()
df_min = df.min()

## Preparing the dataset 

The data is split into training and testing data. x represents the input features whereas y represents the output i.e. the interest rate.As a rule of thumb, we split the data into 80% training data and 20% testing or validation data.

In [ ]:
y = df.iloc[:,df.columns.isin(["int_rate"])]
x = df.loc[:, ~df.columns.isin(["int_rate"])]

total_samples=len(df)
split = 0.8

x_train = x[0:int(total_samples*split)]
x_test = x[int(total_samples*split):total_samples]
y_train = y[0:int(total_samples*split)]
y_test = y[int(total_samples*split):total_samples]

## AutoML

#### TPOT Regressor [link](https://epistasislab.github.io/tpot/api/#regression)
The TPOTRegressor performs an intelligent search over machine learning pipelines that can contain supervised regression models, preprocessors, feature selection techniques, and any other estimator or transformer that follows the scikit-learn API. The TPOTRegressor will also search over the hyperparameters of all objects in the pipeline.

These piplines are various combinations of different preprocessors and sklearn models. Some preprocessors include:
* Binarizer
* FastICA
* FeatureAgglomeration
* MaxAbsScaler
* Normalizer
* PCA
* StandardScaler
* RBFSampler
* OneHotEncoder

TPOT will evaluate POPULATION_SIZE + GENERATIONS x OFFSPRING_SIZE pipelines in total 

**TPOTRegressor 's performance is as good as the amount of time it is allowed to optimize.**

Here is an excerpt from TPOT's official documentation: 

*"TPOT will take a while to run on larger datasets, but it’s important to realize why. With the default TPOT settings (100 generations with 100 population size), TPOT will evaluate 10,000 pipeline configurations before finishing. To put this number into context, think about a grid search of 10,000 hyperparameter combinations for a machine learning algorithm and how long that grid search will take. That is 10,000 model configurations to evaluate with 10-fold cross-validation, which means that roughly 100,000 models are fit and evaluated on the training data in one grid search."*

### The following is all the code needed to find the best model:

In [ ]:
tpot = TPOTRegressor(generations=10, 
                     population_size=10, 
                     verbosity=2, 
                     max_time_mins=5, #total time
                     max_eval_time_mins=1, # time per pipeline
                     scoring='neg_mean_absolute_error')
tpot.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Generation 1 - Current best internal CV score: -0.06293792001605511
Generation 2 - Current best internal CV score: -0.061182967041762716
Generation 3 - Current best internal CV score: -0.061182967041762716
Generation 4 - Current best internal CV score: -0.05976352952758039
Generation 5 - Current best internal CV score: -0.05976352952758039
Generation 6 - Current best internal CV score: -0.059553866944192034
10.21 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: RandomForestRegressor(StandardScaler(input_matrix), bootstrap=False, max_features=0.7500000000000001, min_samples_leaf=12, min_samples_split=6, n_estimators=100)


TPOTRegressor(config_dict=None, crossover_rate=0.1, cv=5,
              disable_update_check=False, early_stop=None, generations=10,
              log_file=<ipykernel.iostream.OutStream object at 0x7f069c139ba8>,
              max_eval_time_mins=1, max_time_mins=10, memory=None,
              mutation_rate=0.9, n_jobs=1, offspring_size=None,
              periodic_checkpoint_folder=None, population_size=10,
              random_state=None, scoring='neg_mean_absolute_error',
              subsample=1.0, template=None, use_dask=False, verbosity=2,
              warm_start=False)

In [ ]:
print(tpot.scoring_function + ": "+ str(tpot.score(x_test,y_test)))

neg_mean_absolute_error: -0.5587674845563538


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Using the best pipeline to make predictions

In [ ]:
prediction = tpot.fitted_pipeline_.predict(x_test)
prediction_train = tpot.fitted_pipeline_.predict(x_train)

In [ ]:
prediction[0:10]

array([14.26382653, 12.42118367, 14.65357143, 10.6486456 ,  7.9       ,
       11.70736752,  5.64592593,  9.94459459, 10.6486456 , 11.52052632])

#### TPOT export
TPOT also allows you to export the best sklearn pipeline obtained from the AutoML flow as a python file using a simple method call.

In [ ]:
tpot.export('tpot_sample.py')

### Export the best model

In [ ]:
import pickle
pickle.dump(tpot.fitted_pipeline_, open('tpot_pipeline.model','wb'))

### MAPE (Mean Absolute Percentage Error)

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
print(y_test.shape, y_train.shape)

(2000, 1) (7999, 1)


In [ ]:
mape_test = mean_absolute_percentage_error(y_test.values.ravel(), prediction)
mape_train = mean_absolute_percentage_error(y_train.values.ravel(), prediction_train)

In [ ]:
print("Training-set MAPE: "+str(mape_train))
print("Test-set MAPE: "+str(mape_test))

Training-set MAPE: 0.2674174583743645
Test-set MAPE: 5.058836822728818


### Actual values

In [ ]:
y_test.values[0:5].ravel()

array([13.49, 11.49, 13.99, 10.59,  7.49])

### Predicted Values

In [ ]:
prediction[0:5]

array([14.26382653, 12.42118367, 14.65357143, 10.6486456 ,  7.9       ])